In [ ]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras import backend as K

num_classes = 10
epochs = 20

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))

def l1_distance(vects):
    x, y = vects
    return K.maximum(K.sum(K.abs(x - y), axis=1, keepdims=True), K.epsilon())


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def l1_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return(shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) +
                  (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


def create_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Conv2D(filters=64, kernel_size = (10, 10), activation='relu')(input)
    x = MaxPooling2D(pool_size = (2,2))(x)
    x = Conv2D(filters = 128, kernel_size = (7,7), activation = 'relu')(x)
    x = MaxPooling2D(pool_size = (2,2))(x)
    x = Conv2D(filters = 128, kernel_size = (4,4), activation = 'relu')(x)
    x = MaxPooling2D(pool_size = (2,2))(x)
    x = Conv2D(filters = 256, kernel_size = (4,4))(x)
    x = Flatten()(x)
    x = Dense(4096, activation ='sigmoid')(x)
    return Model(input, x)


def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [ ]:
# network definition
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(l1_distance,
                  output_shape=l1_dist_output_shape)([processed_a, processed_b])

sig_out = Dense(1, activation='sigmoid')(distance)

model = Model([input_a, input_b], sig_out)
